In [32]:
import numpy as np
import pandas as pd
from os import path
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.callbacks import EarlyStopping
from collections import Counter
import sklearn
from tensorflow.keras.callbacks import LearningRateScheduler

from sklearn import preprocessing, metrics
from sklearn.metrics import (roc_curve, auc, accuracy_score, precision_score, 
                             recall_score, f1_score, balanced_accuracy_score, 
                             matthews_corrcoef)
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, LabelBinarizer
import shap
from imblearn.over_sampling import RandomOverSampler
import innvestigate

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)
np.random.seed(0)

tf.compat.v1.disable_eager_execution()

In [33]:
def confusion_metrics (name_model,predictions,true_labels):

    name = name_model
    pred_label = predictions
    y_test_01 = true_labels 

    with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print(name, file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test_01))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')


    Acc = accuracy_score(y_test_01, pred_label)
    Precision = precision_score(y_test_01, pred_label, average='macro')
    Recall = recall_score(y_test_01, pred_label, average='macro')
    F1 =  f1_score(y_test_01, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test_01, pred_label)
    MCC = matthews_corrcoef(y_test_01, pred_label)


    # voting_acc_01 = Acc
    # voting_pre_01 = Precision
    # weighed_avg_rec_01 = Recall
    # weighed_avg_f1_01 = F1
    # weighed_avg_bacc_01 = BACC
    # weighed_avg_mcc_01 = MCC
    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)

    return Acc, Precision, Recall, F1, BACC, MCC

In [34]:
output_file_name = "CIC_Base.txt"


In [35]:

req_cols = [
    
    ' Packet Length Std', ' Total Length of Bwd Packets', ' Subflow Bwd Bytes',
    ' Destination Port', ' Packet Length Variance', ' Bwd Packet Length Mean',' Avg Bwd Segment Size',
    'Bwd Packet Length Max', ' Init_Win_bytes_backward','Total Length of Fwd Packets',
    ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', ' Average Packet Size', ' Packet Length Mean',
    ' Max Packet Length',
    ' Down/Up Ratio', ' Fwd URG Flags', ' Flow IAT Std', 'Subflow Fwd Packets', ' Flow Packets/s', ' URG Flag Count', 'FIN Flag Count', ' Bwd Packets/s', 'Bwd Avg Bulk Rate'
    , ' act_data_pkt_fwd', ' Fwd Packet Length Std', ' Bwd Avg Bytes/Bulk', ' Active Max', ' Flow IAT Max', ' min_seg_size_forward', ' Bwd Packet Length Std', ' Fwd IAT Std', ' Fwd Avg Bulk Rate', ' Fwd Packet Length Mean', ' Fwd Packet Length Max', ' Idle Std', ' CWE Flag Count', 'Fwd IAT Total'
    
    , ' ACK Flag Count', ' Bwd URG Flags', ' Flow IAT Min', ' Flow IAT Mean', ' Total Backward Packets', ' Fwd Avg Packets/Bulk', 'Fwd Avg Bytes/Bulk', ' SYN Flag Count', ' Min Packet Length', ' Fwd Packet Length Min', 'Idle Mean', 'Fwd PSH Flags', ' Fwd IAT Min'
     
    ,  ' Fwd Header Length', ' RST Flag Count', ' Idle Max', ' PSH Flag Count', ' Bwd Header Length', ' ECE Flag Count', ' Subflow Bwd Packets', 'Active Mean', 'Flow Bytes/s', ' Bwd IAT Mean', ' Avg Fwd Segment Size', ' Bwd Packet Length Min', ' Active Std', ' Bwd IAT Min', ' Flow Duration', 'Fwd Packets/s', ' Fwd IAT Max', 'Bwd IAT Total', ' Idle Min', ' Bwd PSH Flags', ' Bwd Avg Packets/Bulk', ' Total Fwd Packets', ' Active Min', ' Bwd IAT Std', ' Fwd IAT Mean', ' Bwd IAT Max'
            
            , ' Label']

# Information gain top 10 features
top10 = [' Average Packet Size',
          ' Packet Length Std', 
          ' Packet Length Variance', 
          ' Packet Length Mean',
            ' Destination Port', 
            ' Subflow Bwd Bytes', 
            ' Total Length of Bwd Packets', 
            ' Avg Bwd Segment Size', 
            ' Bwd Packet Length Mean',  
            'Bwd Packet Length Max', 
            ' Label']



# req_cols = top10

# req_cols = [' Destination Port',' Flow Duration',' Total Fwd Packets',' Total Backward Packets','Total Length of Fwd Packets',' Total Length of Bwd Packets',' Label']
#---------------------------------------------------------------------
#Load Databases from csv file

path_str = '/home/oarreche@ads.iu.edu/HITL/cicids/cicids_db/'
fraction = 1
#---------------------------------------------------------------------
#Load Databases from csv file
print('---------------------------------------------------------------------------------')
print('Loading Databases')
print('---------------------------------------------------------------------------------')
print('')


df0 = pd.read_csv (path_str + 'Wednesday-workingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)

df1 = pd.read_csv (path_str + 'Tuesday-WorkingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df2 = pd.read_csv (path_str +'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df3 = pd.read_csv (path_str +'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df4 = pd.read_csv (path_str +'Monday-WorkingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df5 = pd.read_csv (path_str +'Friday-WorkingHours-Morning.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df6 = pd.read_csv (path_str +'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df7 = pd.read_csv (path_str +'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)

frames = [df0, df1, df2, df3, df4, df5, df6, df7]

df = pd.concat(frames,ignore_index=True)

df = df.sample(frac = 0.333)



# IG popping unwandted columns descriptive accuracy

# df.pop(' min_seg_size_forward')
# df.pop(' PSH Flag Count')
# df.pop(' ACK Flag Count')
# df.pop('Init_Win_bytes_forward')
# df.pop(' Destination Port')

# df.pop(' Bwd Packet Length Mean')
# df.pop('Fwd Packets/s')
# df.pop(' Init_Win_bytes_backward')
# df.pop(' Packet Length Mean')
# df.pop(' Packet Length Std')

# df.pop(' Bwd Packet Length Std')
# df.pop(' Avg Bwd Segment Size')
# df.pop(' Packet Length Variance')
# df.pop(' Flow Duration')
# df.pop(' Max Packet Length')
# df.pop(' Flow IAT Max')
# df.pop(' Idle Max')
# df.pop('Bwd Packet Length Max')
# df.pop(' Fwd IAT Max')
# df.pop(' URG Flag Count')
# df.pop('Fwd PSH Flags')

# df.pop(' Min Packet Length')
# df.pop('Idle Mean')
# df.pop(' Fwd Packet Length Min')
# df.pop(' Average Packet Size')
# df.pop(' Idle Min')
# df.pop('Bwd IAT Total')
# df.pop('Fwd IAT Total')
# df.pop(' Fwd Packet Length Max')
# df.pop(' Avg Fwd Segment Size')
# df.pop(' SYN Flag Count')
# df.pop(' Fwd Packet Length Mean')
# df.pop(' Flow IAT Std')
# df.pop(' Down/Up Ratio')
# df.pop(' Fwd IAT Mean')
# df.pop('FIN Flag Count')
# df.pop(' Bwd Packets/s')
# df.pop(' Fwd Packet Length Std')
# df.pop(' Flow IAT Mean')
# df.pop(' Bwd Packet Length Min')
# df.pop(' Fwd IAT Std')
# df.pop(' Bwd IAT Mean')

# df.pop(' Bwd IAT Std')
# df.pop(' Bwd IAT Max')
# df.pop(' Bwd IAT Min')
# df.pop(' Fwd IAT Min')
# df.pop(' Idle Std')
# df.pop(' Active Max')
# df.pop(' RST Flag Count')
# df.pop(' Subflow Fwd Bytes')
# df.pop(' ECE Flag Count')
# df.pop(' Flow IAT Min')
# df.pop('Total Length of Fwd Packets')
# df.pop(' Active Std')
# df.pop(' Active Min')
# df.pop(' Bwd Header Length')
# df.pop(' Fwd Header Length')
# df.pop('Active Mean')
# df.pop(' Total Fwd Packets')
# df.pop(' Total Backward Packets')
# df.pop('Subflow Fwd Packets')
# df.pop(' act_data_pkt_fwd')
# df.pop(' Subflow Bwd Bytes')
# df.pop(' Fwd URG Flags')
# df.pop(' CWE Flag Count')
# df.pop(' Subflow Bwd Packets')
# df.pop(' Total Length of Bwd Packets')
# df.pop('Flow Bytes/s')
# df.pop(' Flow Packets/s')
# df.pop(' Bwd PSH Flags')
# df.pop(' Bwd URG Flags')
# df.pop('Fwd Avg Bytes/Bulk')
# df.pop(' Fwd Avg Packets/Bulk')
# df.pop(' Fwd Avg Bulk Rate')
# df.pop(' Bwd Avg Bytes/Bulk')
# df.pop(' Bwd Avg Packets/Bulk')
# df.pop('Bwd Avg Bulk Rate')

# LRP popping unwandted columns descriptive accuracy


# df.pop(' min_seg_size_forward')
# df.pop(' PSH Flag Count')
# df.pop(' ACK Flag Count')
# df.pop('Init_Win_bytes_forward')
# df.pop(' Destination Port')

# df.pop(' Bwd Packet Length Mean')
# df.pop('Fwd Packets/s')
# df.pop(' Init_Win_bytes_backward')
# df.pop(' Packet Length Mean')
# df.pop(' Packet Length Std')

# df.pop(' Bwd Packet Length Std')
# df.pop(' Avg Bwd Segment Size')
# df.pop(' Packet Length Variance')
# df.pop(' Flow Duration')
# df.pop(' Max Packet Length')
# df.pop(' Flow IAT Max')
# df.pop(' Idle Max')
# df.pop('Bwd Packet Length Max')
# df.pop(' Fwd IAT Max')
# df.pop(' URG Flag Count')
# df.pop('Fwd PSH Flags')

# df.pop(' Min Packet Length')
# df.pop('Idle Mean')
# df.pop(' Fwd Packet Length Min')
# df.pop(' Average Packet Size')
# df.pop(' Idle Min')
# df.pop('Bwd IAT Total')
# df.pop('Fwd IAT Total')
# df.pop(' Fwd Packet Length Max')
# df.pop(' Avg Fwd Segment Size')
# df.pop(' SYN Flag Count')
# df.pop(' Fwd Packet Length Mean')
# df.pop(' Flow IAT Std')
# df.pop(' Down/Up Ratio')
# df.pop(' Fwd IAT Mean')
# df.pop('FIN Flag Count')
# df.pop(' Bwd Packets/s')
# df.pop(' Fwd Packet Length Std')
# df.pop(' Flow IAT Mean')
# df.pop(' Bwd Packet Length Min')
# df.pop(' Fwd IAT Std')
# df.pop(' Bwd IAT Mean')
# df.pop(' Bwd IAT Std')

# df.pop(' Bwd IAT Max')
# df.pop(' Bwd IAT Min')
# df.pop(' Fwd IAT Min')
# df.pop(' Idle Std')
# df.pop(' Active Max')
# df.pop(' RST Flag Count')
# df.pop(' Subflow Fwd Bytes')
# df.pop(' ECE Flag Count')
# df.pop(' Flow IAT Min')
# df.pop('Total Length of Fwd Packets')
# df.pop(' Active Std')
# df.pop(' Active Min')
# df.pop(' Bwd Header Length')
# df.pop(' Fwd Header Length')
# df.pop('Active Mean')
# df.pop(' Total Fwd Packets')
# df.pop(' Total Backward Packets')
# df.pop('Subflow Fwd Packets')
# df.pop(' act_data_pkt_fwd')
# df.pop(' Subflow Bwd Bytes')
# df.pop(' Fwd URG Flags')
# df.pop(' CWE Flag Count')
# df.pop(' Subflow Bwd Packets')
# df.pop(' Total Length of Bwd Packets')
# df.pop('Flow Bytes/s')
# df.pop(' Flow Packets/s')
# df.pop(' Bwd PSH Flags')
# df.pop(' Bwd URG Flags')
# df.pop('Fwd Avg Bytes/Bulk')
# df.pop(' Fwd Avg Packets/Bulk')
# df.pop(' Fwd Avg Bulk Rate')
# df.pop(' Bwd Avg Bytes/Bulk')
# df.pop(' Bwd Avg Packets/Bulk')
# df.pop('Bwd Avg Bulk Rate')


# DeepLift popping unwandted columns descriptive accuracy

# df.pop(' PSH Flag Count')
# df.pop(' ACK Flag Count')
# df.pop(' Bwd Packet Length Mean')
# df.pop('Init_Win_bytes_forward')
# df.pop(' Bwd Packet Length Std')

# df.pop('Fwd Packets/s')
# df.pop(' Packet Length Std')
# df.pop(' Packet Length Variance')
# df.pop(' Destination Port')
# df.pop(' Avg Bwd Segment Size')

# df.pop(' Packet Length Mean')
# df.pop(' URG Flag Count')
# df.pop(' Idle Max')
# df.pop(' Flow IAT Max')
# df.pop('Fwd PSH Flags')
# df.pop(' Flow Duration')
# df.pop(' Max Packet Length')
# df.pop('Bwd Packet Length Max')
# df.pop(' Fwd IAT Max')
# df.pop('FIN Flag Count')
# df.pop(' min_seg_size_forward')

# df.pop('Idle Mean')
# df.pop(' Idle Min')
# df.pop(' Init_Win_bytes_backward')
# df.pop('Bwd IAT Total')
# df.pop('Fwd IAT Total')
# df.pop(' Average Packet Size')
# df.pop(' Flow IAT Std')
# df.pop(' Min Packet Length')
# df.pop(' SYN Flag Count')
# df.pop(' Fwd Packet Length Min')
# df.pop(' Fwd IAT Mean')
# df.pop(' Bwd Packet Length Min')
# df.pop(' Fwd IAT Std')
# df.pop(' Fwd Packet Length Max')
# df.pop(' Fwd Packet Length Mean')
# df.pop(' Fwd Packet Length Std')
# df.pop(' Flow IAT Mean')
# df.pop(' Avg Fwd Segment Size')
# df.pop(' Bwd IAT Max')
# df.pop(' Bwd IAT Mean')
# df.pop(' Down/Up Ratio')

# df.pop(' Bwd IAT Min')
# df.pop(' Fwd IAT Min')
# df.pop(' Bwd IAT Std')
# df.pop(' Bwd Packets/s')
# df.pop(' Idle Std')
# df.pop(' Active Max')
# df.pop(' Flow IAT Min')
# df.pop(' Subflow Fwd Bytes')
# df.pop('Total Length of Fwd Packets')
# df.pop(' Active Std')
# df.pop(' RST Flag Count')
# df.pop(' Active Min')
# df.pop('Active Mean')
# df.pop(' ECE Flag Count')
# df.pop(' CWE Flag Count')
# df.pop(' Fwd URG Flags')
# df.pop(' Total Backward Packets')
# df.pop(' act_data_pkt_fwd')
# df.pop(' Total Fwd Packets')
# df.pop(' Subflow Bwd Bytes')
# df.pop('Subflow Fwd Packets')
# df.pop(' Fwd Header Length')
# df.pop(' Bwd Header Length')
# df.pop(' Subflow Bwd Packets')
# df.pop(' Total Length of Bwd Packets')
# df.pop('Bwd Avg Bulk Rate')
# df.pop(' Bwd Avg Packets/Bulk')
# df.pop(' Bwd Avg Bytes/Bulk')
# df.pop(' Fwd Avg Bulk Rate')
# df.pop(' Fwd Avg Packets/Bulk')
# df.pop(' Bwd PSH Flags')
# df.pop('Flow Bytes/s')
# df.pop(' Flow Packets/s')
# df.pop(' Bwd URG Flags')
# df.pop('Fwd Avg Bytes/Bulk')

#---------------------------------------------------------------------
# Normalize database
print('---------------------------------------------------------------------------------')
print('Normalizing database')
print('---------------------------------------------------------------------------------')
print('')

df_max_scaled = df.copy()

y = df_max_scaled[' Label'].replace({'DDoS' :'Dos/Ddos' ,'DoS GoldenEye': 'Dos/Ddos', 'DoS Hulk': 'Dos/Ddos', 'DoS Slowhttptest': 'Dos/Ddos', 'DoS slowloris': 'Dos/Ddos', 'Heartbleed': 'Dos/Ddos','FTP-Patator': 'Brute Force', 'SSH-Patator': 'Brute Force','Web Attack - Brute Force': 'Web Attack', 'Web Attack - Sql Injection': 'Web Attack', 'Web Attack - XSS': 'Web Attack'})

df_max_scaled.pop(' Label')


df_max_scaled
for col in df_max_scaled.columns:
    t = abs(df_max_scaled[col].max())
    df_max_scaled[col] = df_max_scaled[col]/t
df_max_scaled
df = df_max_scaled.assign( Label = y)
#df
df = df.fillna(0)

#---------------------------------------------------------------------

# Separate features and labels 
print('---------------------------------------------------------------------------------')
print('Separating features and labels')
print('---------------------------------------------------------------------------------')
print('')

y = df.pop('Label')
X = df
# summarize class distribution
counter = Counter(y)
print(counter)

df = X.assign( Label = y)

y, label = pd.factorize(y)

# Separate Training and Testing db
print('---------------------------------------------------------------------------------')
print('Separating Training and Testing db')
print('---------------------------------------------------------------------------------')
print('')

X_train,X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=0.7,random_state=42)
df = X.assign( Label = y)




---------------------------------------------------------------------------------
Loading Databases
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Normalizing database
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Separating features and labels
---------------------------------------------------------------------------------

Counter({'BENIGN': 756652, 'Dos/Ddos': 127047, 'PortScan': 52987, 'Brute Force': 4594, 'Web Attack': 716, 'Bot': 628, 'Infiltration': 13})
---------------------------------------------------------------------------------
Separating Training and Testing db
---------------------------------------------------------------------------------



In [36]:

print('---------------------------------------------------------------------------------')
print('Defining the DNN model')
print('---------------------------------------------------------------------------------')
print('')

# dropout_rate = 0.01
nodes = 7

model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(len(X_train.columns,))))

model.add(tf.keras.layers.Dense(nodes, activation='relu'))

model.add(tf.keras.layers.Dense(7))
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])




---------------------------------------------------------------------------------
Defining the DNN model
---------------------------------------------------------------------------------



In [37]:

print('---------------------------------------------------------------------------------')
print('Training the model')
print('---------------------------------------------------------------------------------')
print('')

start = time.time()

early_stopping = EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)

model.fit(X_train, y_train, epochs=1000, batch_size=len(X_train), callbacks=[early_stopping])

end = time.time()
print('---------------------------------------------------------------------------------')
print('ELAPSE TIME TRAINING MODEL: ',(end - start)/60, 'min')
print('---------------------------------------------------------------------------------')
print('')


---------------------------------------------------------------------------------
Training the model
---------------------------------------------------------------------------------

Train on 659845 samples
Epoch 1/1000
659845/659845 [==============================] - 0s 1us/sample - loss: 3.9561 - accuracy: 0.0472
Epoch 2/1000
659845/659845 [==============================] - 0s 1us/sample - loss: 3.1151 - accuracy: 0.3661
Epoch 3/1000
659845/659845 [==============================] - 0s 1us/sample - loss: 2.4446 - accuracy: 0.6511
Epoch 4/1000
659845/659845 [==============================] - 0s 1us/sample - loss: 1.8422 - accuracy: 0.7464
Epoch 5/1000
659845/659845 [==============================] - 0s 1us/sample - loss: 1.7290 - accuracy: 0.7600
Epoch 6/1000
659845/659845 [==============================] - 0s 1us/sample - loss: 1.6461 - accuracy: 0.7960
Epoch 7/1000
659845/659845 [==============================] - 0s 1us/sample - loss: 1.5916 - accuracy: 0.8026
Epoch 8/1000
659845/65

In [38]:

print('---------------------------------------------------------------------------------')
print('Model Prediction')
print('---------------------------------------------------------------------------------')
print('')
print('---------------------------------------------------------------------------------')
start = time.time()
y_pred = model.predict(X_test)
end = time.time()
print('ELAPSE TIME MODEL PREDICTION: ',(end - start)/60, 'min')
print('---------------------------------------------------------------------------------')
print('')

#print(y_pred)
ynew = np.argmax(y_pred,axis = 1)
#print(ynew)
score = model.evaluate(X_test, y_test,verbose=1)
#print(score)
pred_label = label[ynew]

#---------------------------------------------------------------------

accuracy =accuracy_score(y_test, ynew)*100
print('accuracy ',accuracy)

label_counts = Counter(y_test)
print(label_counts)

label_counts = Counter(ynew)
print(label_counts)






---------------------------------------------------------------------------------
Model Prediction
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
ELAPSE TIME MODEL PREDICTION:  0.045297797520955405 min
---------------------------------------------------------------------------------

accuracy  80.23635746414325
Counter({0: 226902, 2: 38054, 1: 16073, 3: 1351, 5: 205, 4: 202, 6: 5})
Counter({0: 282792})


In [39]:
confusion_metrics('dnn', ynew, y_test)


---------------------------------------------------------------------------------
CONFUSION MATRIX
---------------------------------------------------------------------------------
          0    1    2    3    4    5    6
0  226902.0  0.0  0.0  0.0  0.0  0.0  0.0
1   16073.0  0.0  0.0  0.0  0.0  0.0  0.0
2   38054.0  0.0  0.0  0.0  0.0  0.0  0.0
3    1351.0  0.0  0.0  0.0  0.0  0.0  0.0
4     202.0  0.0  0.0  0.0  0.0  0.0  0.0
5     205.0  0.0  0.0  0.0  0.0  0.0  0.0
6       5.0  0.0  0.0  0.0  0.0  0.0  0.0
---------------------------------------------------------------------------------
METRICS
---------------------------------------------------------------------------------
Accuracy total:  0.8023635746414326
Precision total:  0.11462336780591895
Recall total:  0.14285714285714285
F1 total:  0.12719228175560798
BACC total:  0.14285714285714285
MCC total:  0.0


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


(0.8023635746414326,
 0.11462336780591895,
 0.14285714285714285,
 0.12719228175560798,
 0.14285714285714285,
 0.0)

## Integrated Gradients


In [40]:
single_sample_df = X_test.iloc[[0]]

In [41]:
single_sample_df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
2255405,0.000809,0.000775,0.000009,0.000007,0.000055,5.990626e-07,0.001926,0.021792,0.009696,0.0,...,0.000005,0.231884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
%%time
# Create an analyzer for the model
analyzer = innvestigate.create_analyzer("integrated_gradients", model)

analysis = analyzer.analyze(X_test)

# Perform LRP analysis on a certain number of samples
# analysis = analyzer.analyze(X_test.sample(10000))

#uncomment for single sample
# analysis = analyzer.analyze(single_sample_df)

names = X_test.columns

scores = pd.DataFrame(analysis)
scores_abs = scores.abs()

# Calculate the sum of each column
sum_of_columns = scores_abs.sum(axis=0)

names = list(names)

sum_of_columns = list(sum_of_columns)

# Zip the two lists together
combined = list(zip(names, sum_of_columns))

# Sort the combined list in descending order based on the values from sum_of_columns
sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)

# Unzip the sorted_combined list to separate names and sum_of_columns
sorted_names, sorted_sum_of_columns = zip(*sorted_combined)

print(sorted_names)
print(sorted_sum_of_columns)

sorted_names


`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


(' min_seg_size_forward', ' ACK Flag Count', ' URG Flag Count', ' Idle Max', ' Destination Port', ' Packet Length Std', 'Bwd IAT Total', 'Init_Win_bytes_forward', ' Flow IAT Max', ' Flow Duration', 'Fwd IAT Total', ' Avg Bwd Segment Size', ' Idle Min', ' Fwd IAT Max', ' PSH Flag Count', ' Max Packet Length', ' Packet Length Mean', 'Idle Mean', ' Bwd Packet Length Mean', ' SYN Flag Count', 'Bwd Packet Length Max', ' Fwd IAT Std', ' Bwd Packet Length Min', ' Bwd IAT Mean', ' Init_Win_bytes_backward', ' Bwd Packet Length Std', ' Min Packet Length', ' Flow IAT Mean', 'Fwd PSH Flags', ' Fwd IAT Min', 'FIN Flag Count', ' Flow IAT Std', ' Avg Fwd Segment Size', ' Fwd Packet Length Mean', ' Down/Up Ratio', ' Average Packet Size', ' Bwd IAT Max', 'Fwd Packets/s', ' Bwd IAT Std', ' Packet Length Variance', ' Fwd Packet Length Max', ' Fwd Packet Length Std', ' Bwd Packets/s', ' Fwd Packet Length Min', ' Active Max', ' Fwd IAT Mean', ' Idle Std', ' Bwd IAT Min', ' Flow IAT Min', 'Active Mean', ' S

(' min_seg_size_forward',
 ' ACK Flag Count',
 ' URG Flag Count',
 ' Idle Max',
 ' Destination Port',
 ' Packet Length Std',
 'Bwd IAT Total',
 'Init_Win_bytes_forward',
 ' Flow IAT Max',
 ' Flow Duration',
 'Fwd IAT Total',
 ' Avg Bwd Segment Size',
 ' Idle Min',
 ' Fwd IAT Max',
 ' PSH Flag Count',
 ' Max Packet Length',
 ' Packet Length Mean',
 'Idle Mean',
 ' Bwd Packet Length Mean',
 ' SYN Flag Count',
 'Bwd Packet Length Max',
 ' Fwd IAT Std',
 ' Bwd Packet Length Min',
 ' Bwd IAT Mean',
 ' Init_Win_bytes_backward',
 ' Bwd Packet Length Std',
 ' Min Packet Length',
 ' Flow IAT Mean',
 'Fwd PSH Flags',
 ' Fwd IAT Min',
 'FIN Flag Count',
 ' Flow IAT Std',
 ' Avg Fwd Segment Size',
 ' Fwd Packet Length Mean',
 ' Down/Up Ratio',
 ' Average Packet Size',
 ' Bwd IAT Max',
 'Fwd Packets/s',
 ' Bwd IAT Std',
 ' Packet Length Variance',
 ' Fwd Packet Length Max',
 ' Fwd Packet Length Std',
 ' Bwd Packets/s',
 ' Fwd Packet Length Min',
 ' Active Max',
 ' Fwd IAT Mean',
 ' Idle Std',
 ' Bw

## LRP


In [43]:
%%time

# Create an analyzer for the model
analyzer = innvestigate.create_analyzer("lrp.z", model)

# Perform LRP analysis on the input data
analysis = analyzer.analyze(X_test)

# Perform LRP analysis on a certain number of samples
# analysis = analyzer.analyze(X_test.sample(10000))

#uncomment for single sample
# analysis = analyzer.analyze(single_sample_df)


names = X_test.columns
scores = pd.DataFrame(analysis)
scores_abs = scores.abs()

# Calculate the sum of each column
sum_of_columns = scores_abs.sum(axis=0)

names = list(names)

sum_of_columns = list(sum_of_columns)

# Zip the two lists together
combined = list(zip(names, sum_of_columns))

# Sort the combined list in descending order based on the values from sum_of_columns
sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)

# Unzip the sorted_combined list to separate names and sum_of_columns
sorted_names, sorted_sum_of_columns = zip(*sorted_combined)

print(sorted_names)
print(sorted_sum_of_columns)

sorted_names


(' min_seg_size_forward', ' ACK Flag Count', ' URG Flag Count', ' Idle Max', ' Packet Length Std', 'Init_Win_bytes_forward', 'Bwd IAT Total', ' Destination Port', ' Flow IAT Max', ' PSH Flag Count', ' Avg Bwd Segment Size', ' Flow Duration', 'Fwd IAT Total', ' Idle Min', ' Fwd IAT Max', ' Max Packet Length', ' Bwd Packet Length Mean', ' Packet Length Mean', 'Idle Mean', ' SYN Flag Count', 'Bwd Packet Length Max', ' Fwd IAT Std', ' Bwd IAT Mean', ' Init_Win_bytes_backward', ' Bwd Packet Length Min', ' Min Packet Length', ' Fwd IAT Min', 'Fwd PSH Flags', ' Flow IAT Mean', ' Bwd Packet Length Std', 'FIN Flag Count', ' Avg Fwd Segment Size', ' Fwd Packet Length Mean', ' Down/Up Ratio', ' Flow IAT Std', ' Average Packet Size', ' Bwd IAT Max', ' Bwd IAT Std', ' Packet Length Variance', 'Fwd Packets/s', ' Fwd Packet Length Max', ' Bwd Packets/s', ' Fwd Packet Length Std', ' Fwd IAT Mean', ' Fwd Packet Length Min', ' Active Max', ' Bwd IAT Min', ' Idle Std', ' Flow IAT Min', 'Active Mean', ' S

(' min_seg_size_forward',
 ' ACK Flag Count',
 ' URG Flag Count',
 ' Idle Max',
 ' Packet Length Std',
 'Init_Win_bytes_forward',
 'Bwd IAT Total',
 ' Destination Port',
 ' Flow IAT Max',
 ' PSH Flag Count',
 ' Avg Bwd Segment Size',
 ' Flow Duration',
 'Fwd IAT Total',
 ' Idle Min',
 ' Fwd IAT Max',
 ' Max Packet Length',
 ' Bwd Packet Length Mean',
 ' Packet Length Mean',
 'Idle Mean',
 ' SYN Flag Count',
 'Bwd Packet Length Max',
 ' Fwd IAT Std',
 ' Bwd IAT Mean',
 ' Init_Win_bytes_backward',
 ' Bwd Packet Length Min',
 ' Min Packet Length',
 ' Fwd IAT Min',
 'Fwd PSH Flags',
 ' Flow IAT Mean',
 ' Bwd Packet Length Std',
 'FIN Flag Count',
 ' Avg Fwd Segment Size',
 ' Fwd Packet Length Mean',
 ' Down/Up Ratio',
 ' Flow IAT Std',
 ' Average Packet Size',
 ' Bwd IAT Max',
 ' Bwd IAT Std',
 ' Packet Length Variance',
 'Fwd Packets/s',
 ' Fwd Packet Length Max',
 ' Bwd Packets/s',
 ' Fwd Packet Length Std',
 ' Fwd IAT Mean',
 ' Fwd Packet Length Min',
 ' Active Max',
 ' Bwd IAT Min',
 '

## SHAP


In [44]:
%%time

with open(output_file_name, "a") as f:print('',file = f)


print('---------------------------------------------------------------------------------')
print('Generating Explainer')
print('---------------------------------------------------------------------------------')

samples = 2500

# uncomment for single sample
# samples = 1

Label = label

test = X_test
train = X_train
start_index = 0

end_index = samples
explainer = shap.DeepExplainer(model,train[start_index:end_index].values.astype('float'))
shap_values = explainer.shap_values(test[start_index:end_index].values.astype('float'))

vals= np.abs(shap_values).mean(1)
feature_importance = pd.DataFrame(list(zip(train.columns, sum(vals))), columns=['col_name','feature_importance_vals'])
feature_importance.sort_values(by=['feature_importance_vals'], ascending=False,inplace=True)
feature_importance.head()
print(feature_importance.to_string())



print('---------------------------------------------------------------------------------')
# feature_importance_vals = 'feature_importance_vals'  # Replace with the name of the column you want to extract
feature_val = feature_importance['feature_importance_vals'].tolist()

# col_name = 'col_name'  # Replace with the name of the column you want to extract
feature_name = feature_importance['col_name'].tolist()

# Use zip to combine the two lists, sort based on list1, and then unzip them
zipped_lists = list(zip(feature_name, feature_val))
zipped_lists.sort(key=lambda x: x[1],reverse=True)

# Convert the sorted result back into separate lists
sorted_list1, sorted_list2 = [list(x) for x in zip(*zipped_lists)]

for k in sorted_list1:
  with open(output_file_name, "a") as f:print("df.pop('",k,"')", sep='',file = f)
with open(output_file_name, "a") as f:print("Trial_ =[", file = f)
print("Trial_ =[")
for k in sorted_list1:
  with open(output_file_name, "a") as f:print("'",k,"',", sep='', file = f)
  print("'",k,"',", sep='')
with open(output_file_name, "a") as f:print("]", file = f)
print('---------------------------------------------------------------------------------')




---------------------------------------------------------------------------------
Generating Explainer
---------------------------------------------------------------------------------


Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


                        col_name  feature_importance_vals
47                ACK Flag Count                 0.523299
46                PSH Flag Count                 0.180458
0               Destination Port                 0.166056
18                  Flow IAT Max                 0.163436
65        Init_Win_bytes_forward                 0.156911
20                 Fwd IAT Total                 0.153937
48                URG Flag Count                 0.105572
73                     Idle Mean                 0.091728
12        Bwd Packet Length Mean                 0.081705
40            Packet Length Mean                 0.077650
25                 Bwd IAT Total                 0.075862
75                      Idle Max                 0.059529
66       Init_Win_bytes_backward                 0.058012
1                  Flow Duration                 0.054205
23                   Fwd IAT Max                 0.050449
41             Packet Length Std                 0.048170
10         Bwd